<br>
<strong><center>Instituto Superior de Engenharia de Lisboa</center></strong>
<center>Licenciatura em Engenharia Informática e Multimédia</center>
<strong><center>Codificação de Sinais Multimédia</center></strong>

<strong><center>2º Semestre de 2018/2019</center></strong>
<br>

## <center>2º Trabalho Prático</center>

<center><strong><em>Autores:</em></strong></center>
<center><em>Luís Guimarães 42346</em></center>
<center><em>Érica Pereira 42356</em></center>


<br>
<center><strong><em>Docente:</em></strong></center>
<center><em>André Lourenço</em></center>

<br>
<br>
<div style='text-align: right; margin-right:10%;'><strong>Lisboa, 14 de abril de 2019</strong></div>

In [1]:
# imports
import cv2 
import numpy as np
import matplotlib.pyplot as plt 

from os import path 
from time import time

import os
cwd = os.getcwd()

print(cwd)
#lena.tif esta no cwd

def waitKey():
    key = cv2.waitKey(10000)
    if key == 27:
        cv2.destroyAllWindows()


C:\Users\erica\Desktop\ISEL\2a02 - CSM\4ano\Trabalho 2


## Exercício 1

### Huffman

O método de Huffman é um método de compressão sem perdas, ou seja, reduz o número de bits necessários para representar cada símbolo sem perda de informação neste processo de compressão. 
O código gerado para cada símbolo é de comprimento variável, baseado numa codificação entrópica. Os símbolos com maior número de ocorrências de um dado conjunto são representados por menos bits do que os símbolos com um menor número de ocorrências.
<br>

O algoritmo teórico do método de Huffman dá-se da seguinte forma:
<br>
1. Colocar os símbolos numa lista por ordem decrescente de probabilidade.
<br>
2. Juntar os dois símbolos com menor número de ocorrência num novo símbolo, com a respectiva probabilidade igual à soma das probabilidades dos dois.
<br>
3. Inserir o novo símbolo na lista ordenada.
<br>
4. Voltar ao ponto 2 até que a lista contenha apenas um símbolo.
<br>
5. Percorrendo as listagens das junções de símbolos, desde a lista do último símbolo criado até à lista dos símbolos originais, atribuir números binários <br> (0 ou 1) aos símbolos que vão sendo separados, distinguindo um simbolo do outro.
<br>

A abordagem tomada neste trabalho, sucedeu-se de maneira a codificar os símbolos à medida que se vão dando as junções dos símbolos.

### Função <em>listagem_ocorrencias</em>

A função que implementa o método de Huffman vai ter como parâmetros de entrada um conjunto de símbolos e o número de ocorrências de cada símbolo. Para receber os parâmetros de entrada no formato pretendido, fez-se a função <em>listagem_ocorrencias</em>, que recebe um dado conjunto de simbolos de um ficheiro e devolve um array de símbolos não repetidos e um array com os respetivos números de ocorrência. Ambos os arrays devolvidos já se encontram ordenados de forma decrescente.
 

In [2]:

def listagem_ocorrencias(mensagem):
    
    simbolos = np.unique(mensagem) 
    numSimbolos = len(simbolos)
    
    ocorrencias = {}
    
    #percorre cada elemento da mensagem e incrementa o seu número de ocorrências no dicionário
    for simbolo in mensagem:
        if simbolo in ocorrencias:
            ocorrencias[simbolo] += 1
        else:
            ocorrencias[simbolo] = 1
        
    # Organiza os símbolos segundo as suas ocorrências de modo decrescente (maior frequência primeiro)
    sortedOcorrencias = sorted(ocorrencias.items(), key = lambda simbolo: simbolo[1], reverse = True)
    
    #retorna um array de simbolos e um array das respectivas ocorrencias, os dois ordenados
    return [simbolo for (simbolo, frequencia) in sortedOcorrencias], [frequencia for (simbolo, frequencia) in sortedOcorrencias]


# ------------------------- teste -------------------------
mensagem = np.fromfile("otorrino.txt", dtype = np.uint8) #conteúdo: "otorrinolaringologista"
print("File: ", mensagem)
conjunto_simbolos, ocorrencias = listagem_ocorrencias(mensagem)
print("conjunto_simbolos: " + str(conjunto_simbolos))
print("ocorrencias: " + str(ocorrencias))
print()

File:  [111 116 111 114 114 105 110 111 108  97 114 105 110 103 111 108 111 103
 105 115 116  97]
conjunto_simbolos: [111, 114, 105, 116, 110, 108, 97, 103, 115]
ocorrencias: [5, 3, 3, 2, 2, 2, 2, 2, 1]



### Função <em>gera_huffman</em>

A função <em>gera_huffman</em> gera uma tabela com um código binário para cada símbolo de um dado conjunto, usando o método de Huffman. Tem como parâmetros de entrada um conjunto de símbolos e o número de ocorrências de cada símbolo.
<br>

Para a realização do processo do Huffman, foi necessário os seguintes componentes:

 - <strong>codigosBinary/binary</strong> - é um array de bytes vazio do tamanho do número de símbolos existente, onde vão sendo adicionados 0's e 1's nos índices/posições respectivas dos símbolos, codificando-os à medida que se vão juntando. Esta posição/índice vai ser atribuída a cada símbolo no dicionário <em>indexes</em>;
 <br>
 
 - <strong>indexes</strong> - é um dicionário auxiliar que contém como 'keys' um np.arange, que atribui um índice a cada símbolo, e contém, como 'values', as respectivas ocorrências. <br>
 Este dicionário vai servir para ordenar as posições dos símbolos, atualizados com as junções, enquanto ordena as ocorrências, que são fáceis de ordenar por serem inteiros. <br>
 As 'keys' do dicionário são índices em vez dos próprios símbolos porque vão representar a posição do respetivo símbolo no array binary (servindo diretamente como um índice);
 <br>
 
 - <strong>aux</strong> - é um array bidimensional (array de arrays) que vai conter cada índice do respetivo símbolo dentro de um array, como por exemplo: <br><code>[['0'], ['1'], ['2']]</code>. <br> 
 Este array de arrays serve, caso haja índices que ultrapassem 1 dígito, para distinguir o número de dígitos que o índice tem, como por exemplo: <br><code>[['0'], ['1'], ['2', '10']]</code> em vez de <code>['0', '1', '210']</code>. Este último array iria gerar código nos índices '2', '1' e '0' do array <em>binary</em>, em vez de gerar nos índices pretendidos - '2' e '10';
 <br>
 
 - <strong>s</strong> - é um array dos símbolos atualizados com as junções. Vai servir de auxílio na manipulação das junções de símbolos do array <em>indexes</em>;
  <br>
 
 - <strong>o</strong> - é um array das ocorrências atualizadas com as junções. Irá também servir de auxílio na manipulação das somas de ocorrências do array <em>indexes</em>;
  <br>
 
 - <strong>tabela</strong> - é o dicionário final com os símbolos iniciais como 'keys' e as suas respectivas codificações como 'values'.

<br>
Para acompanhar/visualizar o processo de Huffman, foi feita a função <em>printThroughHuffman</em>, que vai imprimindo na consola os arrays e dicionários envolvidos no processo (descomentar as duas chamadas da função no método Huffman).
<br>
<br>

In [3]:
def gera_huffman(conjunto_simbolos, ocorrencias):
    
    #array onde vão ser guardadas as codificações dos símbolos
    codigosBinary = binary = [b'' for indexSimbolo in np.arange(0, len(conjunto_simbolos))]
    
    #dicionario auxiliar para ordenar os simbolos a partir das suas ocorrências
    indexes = {str(simbolo): ocorrencia for simbolo, ocorrencia in zip(np.arange(0, len(conjunto_simbolos)), ocorrencias)}
    
    #array auxiliar em que cada elemento é um array de cada símbolo onde se vão juntar os símbolos de menor frequência
    aux = [[k] for k in list(indexes.keys())]
    
    #enquanto o número de símbolos for maior que dois, agrupam-se os dois símbolos com menor frequência
    while(len(indexes) > 1): 
        
        s = list(indexes.keys())        #simbolos actualizados com a nova ordem
        o = list(indexes.values())      #ocorrencias actualizadas com a nova ordem
     
        #printThroughHuffman(indexes, s, o, aux, binary)
        
        
        #atribui o código binário aos 2 símbolos (ou aos 2 agregados de símbolos) com menor frequência 
        #(um for para atribuir '0' ao primeiro agregado de símbolos e um outro for para atribuir '1' ao segundo)
        for indice in aux[len(aux)-2]:
            binary[int(indice)] = b'0' + binary[int(indice)]
            
        for indice in aux[len(aux)-1]:
            binary[int(indice)] = b'1' + binary[int(indice)]
            #junta os últimos símbolos/agregado de símbolos (os com menor frequência)
            aux[len(aux)-2].append(indice)           
    
        #remove o último elemento do array porque foi agrupado ao penúltimo elemento
        aux = aux[:-1]
           
        #remove os símbolos que foram agrupados, para posteriormente guardar-se o novo símbolo com a nova
        #respectiva ocorrência
        indexes.pop(s[len(s)-2], None)
        indexes.pop(s[len(s)-1], None)
        
        
        #atualiza os novos simbolos(agrupados) e as novas ocorrências (as somas de probabilidades) e
        #remove o último elemento do array (que foi agrupado ao penúltimo elemento)
        s[len(s)-2] = s[len(s)-2] + s[len(s)-1] 
        s = s[:-1]

        o[len(o)-2] = o[len(o)-2] + o[len(o)-1]
        o = o[:-1]

        #guardar-se o novo símbolo com a nova respectiva ocorrência
        indexes[s[len(s)-1]] = o[len(o)-1]
        
        indexes = sorted(indexes.items(), key = lambda simbolo: simbolo[1], reverse = True)
        #transforma de lista ordenada novamente para um dicionário
        indexes = {simbolo: ocorrencia for simbolo, ocorrencia in indexes}
        
        
        # ---> Processo de ordenação dos símbolos no array aux <---
        
        #transforma a lista de símbolos do dicionário num array
        keys = np.array(list(indexes.keys()))
        
        #devolve um array de booleanos 'a', onde fica a True na posição onde um símbolo do array 'keys'
        #for igual ao último elemento do array dos símbolos 's' (s[len(s)-1] -> a última junção de simbolos feita)
        a = keys == s[len(s)-1]
        
        #np.where(array de bools) - devolve os índices das posições que estão a True
        index = np.where(a) #ou index = np.where(a == True)
        
        #vai inserir no index (dois indices porque é um tuplo) o simbolo pretendido, e fazer um "shift"
        #nos indices seguintes do array aux, sem substituir o simbolo nesse index
        aux.insert(index[0][0], aux[len(aux)-1])
        aux = aux[:-1]
    
    #printThroughHuffman(indexes, s, o, aux, binary)
    
    tabela = {simbolo: bits for simbolo, bits in zip(conjunto_simbolos, binary)}
    
    return tabela


# ------------------------- teste -------------------------
#visualizar o processo do huffman (de preferência com um exemplo pequeno)
def printThroughHuffman(indexes, s, o, aux, binary):
    print("size: " + str(len(indexes)))
    print("simbolos actualizados: " + str(s))
    print("ocorrencias actualizadas: " + str(o))
    print("indexes: " + str(indexes))
    print("aux: " + str(aux))
    print("binary: " + str(binary))
    print("--------------------------------------------")
    print()
    
mensagem = open("otorrino.txt", 'rb')
mensagem = mensagem.read()
huffman = gera_huffman(conjunto_simbolos, ocorrencias)
print("Algoritmo de Huffman - Tabela de codificação: ")
print(huffman)
#for h,r in huffman.items():
#    print(r,h)

Algoritmo de Huffman - Tabela de codificação: 
{111: b'10', 114: b'001', 105: b'010', 116: b'0000', 110: b'0001', 108: b'110', 97: b'111', 103: b'0110', 115: b'0111'}
